In [ ]:
import pandas as pd
import numpy as np
pd.options.plotting.backend = 'plotly'

In [ ]:
feb_vacc_data = pd.read_excel('../data/covid-19-vaccination-vaccination-data-28-february-2022.xlsx', index_col=0)['Value']
dec_vacc_data = pd.read_excel('../data/covid-19-vaccination-vaccination-data-22-december-2022.xlsx', index_col=0)['Value']

In [ ]:
agegroups = ['16-19'] + [f'{age}-{age + 4}' for age in range(20, 95, 5)] + ['95+']
doses = ['1+', '2+']
vacc_dict = {}
for dose in doses:
    s = 's' if dose == '2+' else ''
    vacc_dict[dose] = dec_vacc_data[[f'Age group - {age} - Number of people who have received at least {dose[0]} dose{s}' for age in agegroups]].values
final_vacc_data = pd.DataFrame(vacc_dict, index=agegroups)
final_vacc_data['pop'] = dec_vacc_data[[f'Age group - {age} - Population' for age in agegroups]].values

In [ ]:
age_strata = list(range(0, 80, 5))
modelled_vacc = pd.DataFrame(0.0, index=age_strata, columns=doses)
for dose in doses:
    modelled_vacc[dose][15] = final_vacc_data[dose]['16-19'] / final_vacc_data['pop']['16-19']
    for age in age_strata[4: -1]:
        modelled_vacc[dose][age] = final_vacc_data[dose][f'{age}-{age + 4}'] / final_vacc_data['pop'][f'{age}-{age + 4}']
    modelled_vacc[dose][75] = final_vacc_data.loc['75-79':, '1+'].sum() / final_vacc_data.loc['75-79':, 'pop'].sum()
    modelled_vacc[dose][modelled_vacc[dose] > 1.0] = 1.0
    s = 's' if dose == '2+' else ''
    modelled_vacc[dose][5] = dec_vacc_data[f'National - Number of people 5-11 who have received at least {dose[0]} dose{s}'] / dec_vacc_data['National - Population 5-11']
    modelled_vacc[dose][10] = dec_vacc_data[f'National - Number of people 12-15 who have received at least {dose[0]} dose{s}'] / dec_vacc_data['National - Population 12-15']

In [ ]:
modelled_vacc.plot()

In [ ]:
agegroups = ['16-19'] + [f'{age}-{age + 4}' for age in range(20, 95, 5)] + ['95+']
doses = ['1+', '2+']
vacc_dict = {}
vacc_dict['1+'] = feb_vacc_data[[f'Age group - {age} - Number of people with 1 dose' for age in agegroups]].values
final_vacc_data['pop'] = feb_vacc_data[[f'Age group - {age} - Population' for age in agegroups]].values
missing_space_agegroups = ['16-19 '] + [f'{age}-{age + 4} ' for age in range(20, 95, 5)] + ['95+']
vacc_dict['2+'] = feb_vacc_data[[f'Age group - {age}- Number of people fully vaccinated' for age in missing_space_agegroups]].values
final_vacc_data = pd.DataFrame(vacc_dict, index=agegroups)
final_vacc_data['pop'] = feb_vacc_data[[f'Age group - {age} - Population' for age in agegroups]].values

In [ ]:
age_strata = list(range(0, 80, 5))
feb_modelled_vacc = pd.DataFrame(0.0, index=age_strata, columns=doses)
for dose in doses:
    feb_modelled_vacc[dose][15] = final_vacc_data[dose]['16-19'] / final_vacc_data['pop']['16-19']
    for age in age_strata[4: -1]:
        feb_modelled_vacc[dose][age] = final_vacc_data[dose][f'{age}-{age + 4}'] / final_vacc_data['pop'][f'{age}-{age + 4}']
    feb_modelled_vacc[dose][75] = final_vacc_data.loc['75-79':, '1+'].sum() / final_vacc_data.loc['75-79':, 'pop'].sum()
    feb_modelled_vacc[dose][feb_modelled_vacc[dose] > 1.0] = 1.0
feb_modelled_vacc['1+'][5] = feb_vacc_data[f'National - Number of people 5-11 with 1 dose'] / feb_vacc_data['National - Population 5-11']
feb_modelled_vacc['2+'][5] = feb_vacc_data[f'National - Number of people 5-11 fully vaccinated'] / feb_vacc_data['National - Population 5-11']
# modelled_vacc['1+'][10] = feb_vacc_data[f'National - Number of people 12-15 with 1 dose'] / feb_vacc_data['National - Population 12-15']
# modelled_vacc['2+'][10] = feb_vacc_data[f'National - Number of people 12-15 fully vaccinated'] / feb_vacc_data['National - Population 12-15']

In [ ]:
feb_modelled_vacc.columns = [f'feb_{col}' for col in feb_modelled_vacc.columns]

In [ ]:
pd.concat([modelled_vacc, feb_modelled_vacc], axis=1).plot()